In [31]:
# MAIN
#
# # Code to connect Jupyter Notebook in Google Colab
# from google.colab import drive
# drive.mount('/content/drive')
#

In [32]:
# import sys
# sys.path.append('/content/drive/My Drive/\
# ColabNotemodules/funds-explorer-filter')
# # To check the modules are being reacheble by this code (linux command)
# # !ls /content/drive/My\ Drive/ColabNotemodules/funds-explorer-filter/src
#

In [33]:
# packages: urllib, beatiful soup, html5, pandas, plotly, jupyter lab
#
import os
from src.webtableparser import WebTableParser
from src.fundsexplorer import process_FE
from src.yahoofinance import costPrice_YF
import plotly.offline as py
import plotly.graph_objs as go
from datetime import datetime
from time import sleep
from pytz import timezone
#

In [34]:
# running time reading
dt = datetime.now()
tz = timezone('America/Sao_Paulo')
datetime_loc = dt.astimezone(tz)
dt_loc = datetime_loc.strftime('%d/%m/%Y %H:%M')
dt_loc_file = datetime_loc.strftime('%d%m%Y')

In [35]:
# site capture and parsing
site = WebTableParser()
site.create('https://www.fundsexplorer.com.br/ranking',
            'id', 'table-ranking')
table = site.capture()
df = site.parse(table)
#

In [36]:
# real state fund varible, df processing to make analysis feasible
rsf = process_FE(df)
del df
#

In [37]:
# python notebook activation
py.init_notebook_mode(connected=True)
#

In [38]:
# VALUATION STRATEGY - real state funds (rsf) dataframe manipulation
# treasury bonds DY% with IRS (IRRF) tax
t_bond = 2.0 * (1-0.175)
#

In [39]:
# MY FUNDS - BUYED ASSETS - specific assets choosen by user in my_rsf
my_rsf = rsf.copy()
# change the assets if necessary in the next line
my_rsf = my_rsf.loc[(rsf['codigo'] == 'HGRU11') |
                    (rsf['codigo'] == 'XPLG11') |
                    (rsf['codigo'] == 'VISC11') |
                    (rsf['codigo'] == 'SADI11') |
                    (rsf['codigo'] == 'HFOF11')]
# in the case of buyed assets, insert the real prices in dataframe
# add or exclude the asset price average aquisition cost in next lines
my_rsf.loc[my_rsf['codigo'] == 'HGRU11', 'precocustoR$'] = 132.15
my_rsf.loc[my_rsf['codigo'] == 'XPLG11', 'precocustoR$'] = 128.28
my_rsf.loc[my_rsf['codigo'] == 'VISC11', 'precocustoR$'] = 110.57
my_rsf.loc[my_rsf['codigo'] == 'SADI11', 'precocustoR$'] = 102.00
my_rsf.loc[my_rsf['codigo'] == 'HFOF11', 'precocustoR$'] = 118.67
#

In [40]:
# MY FUNDS SCENARIO - time in months valuating the asset
for rsf_time in range(12, 24, 12):
    # my funds % performance over treasury bond calculation
    my_rsf['perf%'] = ((my_rsf['precoatualR$']
                        * (1 + ((rsf_time / 6) * (rsf['dy6macum%'] / 100))))
                       / (my_rsf['precocustoR$']
                       * (1 + ((rsf_time / 6) * (t_bond / 100)))) - 1) * 100
    my_rsf = my_rsf.sort_values(by='perf%', ascending=False)
    # bar chart 0 - MY FUNDS
    x0 = [my_rsf['setor'], my_rsf['codigo']]
    trace00 = go.Bar(x=x0, y=my_rsf['perf%'],
                     name='Perf%=((Pr.At+DY)/(Pr.Ct+RF)-1)',
                     marker_color='rgb(70, 130, 180)')
    trace02 = go.Bar(x=x0, y=my_rsf['dy12macum%'],
                     name='DY% Ano', marker_color='rgb(135, 206, 250)')
    trace04 = go.Bar(x=x0, y=my_rsf['p/vpaN'],
                     name='P/VPA', marker_color='rgb(176, 196, 222)')
    trace06 = go.Bar(x=x0, y=my_rsf['vacfisica%'],
                     name='%Vacância Física',
                     marker_color='rgb(192, 192, 192)')
    trace08 = go.Bar(x=x0, y=my_rsf['varpatr%'],
                     name='%Var. Patr. Acum',
                     marker_color='rgb(211, 211, 211)')
    data0 = [trace00, trace02, trace04, trace06, trace08]
    fig0 = go.Figure(data0)
    fig0.update_layout(title='MEUS FIIs')
    py.plot(fig0)
    fig0.show()  # to use with Jupyter Notebook
    print(f'N DYs = {rsf_time} meses')
    print(dt_loc)
    sleep(8)
    # rename and move tmp files files generated by plotly
    os.rename('temp-plot.html',
              'C:/Users/maber/OneDrive/Documentos/3_Financas/FIIs/myrsf_'
              + str(rsf_time) + 'm_' + dt_loc_file+'.html')
#

N DYs = 12 meses
19/11/2020 16:53


In [41]:
# real state funds (rsf) dataframe in general being filtered by criteria
rsf = rsf.loc[rsf['dy12macum%'] >= 3.00]  # 1st filter DY >= 3%
rsf = rsf.loc[rsf['patrliqR$'] >= 500000000.00]  # 2nd filter > BRL 500 M
rsf = rsf.loc[rsf['liqdiariaNeg'] >= 1000]  # 3rd filter tradings >= 1000/day
rsf = rsf.loc[rsf['p/vpaN'] <= 1.25]  # 4th filter P/VPA <= 1.25
rsf = rsf.loc[rsf['vacfisica%'] <= 15]  # 5th filter vacancy =< 15%

In [42]:
# adding column to a target cost price - average minus 1 standard deviation
# in case of NaN, cost price filled with 90% of current price
df = rsf.copy()
rsf = costPrice_YF(df, 200)
del df

RECR11.SA : PrCusto = 90% PrAtual
CPTS11.SA : PrCusto = 90% PrAtual


In [43]:
# added column in dataframe with performance
rsf['overprice%'] = round(((rsf['precoatualR$'] /
                          rsf['precocustoR$'] - 1) * 100), 2)

In [44]:
# added column in performance% and opportunity%
rsf['perf%'] = 0.0
rsf['oppt%'] = 0.0

In [45]:
# splitting into brick and paper funds
rsf_brick = rsf.copy()
rsf_brick = rsf_brick.loc[rsf['qtdativosN'] >= 5]  # 6th filter >= 5 assets
rsf_paper = rsf.copy()
rsf_paper = rsf_paper.loc[rsf['qtdativosN'] == 0]  # 6th filter = 0 assets
#

In [46]:
# BRICK FUNDS SCENARIO - time in months valuating the asset
for rsf_time in range(12, 24, 12):
    rsf_brick['perf%'] = round(((rsf_brick['precoatualR$']
                                * (1 + ((rsf_time / 6)
                                 * (rsf['dy6macum%'] / 100))))
                                / (rsf_brick['precocustoR$']
                                * (1 + ((rsf_time / 6)
                                   * (t_bond / 100)))) - 1) * 100, 2)
    rsf_brick['oppt%'] = rsf_brick['perf%'] - rsf['overprice%']
    rsf_brick = rsf_brick.sort_values(by='oppt%', ascending=False)
    if rsf_time == 12:
        rsf_brick.to_excel(
            'C:/Users/maber/OneDrive/Documentos/3_Financas/FIIs/rsf_brick'
            + dt_loc_file + '.xlsx',
            columns=['codigo', 'setor', 'oppt%', 'perf%', 'overprice%',
                     'dy12macum%', 'p/vpaN', 'vacfisica%', 'precoatualR$',
                     'precocustoR$'], float_format="%.2f")
        # print(rsf_brick)
    # bar chart 1 - BRICK FUNDS
    x1 = [rsf_brick['setor'], rsf_brick['codigo']]
    trace10 = go.Bar(x=x1, y=rsf_brick['oppt%'],
                     name='Oportunidade%',
                     marker_color='rgb(0, 255, 255)')
    trace11 = go.Bar(x=x1, y=rsf_brick['perf%'],
                     name='Perf%=((Pr.At+DY)/(Pr.Ct+RF)-1)',
                     marker_color='rgb(70, 130, 180)')
    trace12 = go.Bar(x=x1, y=rsf_brick['overprice%'],
                     name='OverPrice%=((Pr.At)/(Pr.Ct)-1)',
                     marker_color='rgb(176, 196, 222)')
    trace13 = go.Bar(x=x1, y=rsf_brick['dy12macum%'],
                     name='DY% Ano', marker_color='rgb(135, 206, 250)')
    trace14 = go.Bar(x=x1, y=rsf_brick['p/vpaN'],
                     name='P/VPA', marker_color='rgb(150, 150, 150)')
    trace16 = go.Bar(x=x1, y=rsf_brick['vacfisica%'],
                     name='%Vacância Física',
                     marker_color='rgb(192, 192, 192)')
    data1 = [trace10, trace11, trace12, trace13, trace14, trace16]
    fig1 = go.Figure(data1)
    fig1.update_layout(title='ANÁLISE FIIs TIJOLOS | Pr.Custo=Pr.Medio200d-Des.Padrão,\
 DYAno>=3%, Patr.>500M, Neg/dia>1000, P/VPA =< 1.25, Ativos >= 5,\
 Vac.Física < 15%')
    py.plot(fig1)
    fig1.show()  # to use with Jupyter Notebook
    print(f'N DYs = {rsf_time} meses')
    print(dt_loc)
    sleep(8)
    # rename and move tmp files files generated by plotly
    os.rename('temp-plot.html',
              'C:/Users/maber/OneDrive/Documentos/3_Financas/FIIs/rsf_brick'
              + str(rsf_time) + 'm_' + dt_loc_file+'.html')
#

N DYs = 12 meses
19/11/2020 16:53


In [47]:
# PAPER FUNDS SCENARIO - time in months valuating the asset
for rsf_time in range(12, 24, 12):
    rsf_paper['perf%'] = ((rsf_paper['precoatualR$']
                           * (1 + ((rsf_time / 6) * (rsf['dy6macum%'] / 100))))
                          / (rsf_paper['precocustoR$']
                          * (1 + ((rsf_time / 6) * (t_bond / 100)))) - 1) * 100
    rsf_paper['oppt%'] = rsf_paper['perf%'] - rsf_paper['overprice%']
    rsf_paper = rsf_paper.sort_values(by='oppt%', ascending=False)
    if rsf_time == 12:
        rsf_paper.to_excel(
            'C:/Users/maber/OneDrive/Documentos/3_Financas/FIIs/rsf_paper'
            + dt_loc_file + '.xlsx',
            columns=['codigo', 'setor', 'oppt%', 'perf%', 'overprice%',
                     'dy12macum%', 'p/vpaN', 'varpatr%', 'precoatualR$',
                     'precocustoR$'], float_format="%.2f")
    # bar chart 2 - PAPER FUNDS
    x2 = [rsf_paper['setor'], rsf_paper['codigo']]
    trace20 = go.Bar(x=x2, y=rsf_paper['oppt%'],
                     name='Oportunidade%',
                     marker_color='rgb(0, 255, 255)')
    trace21 = go.Bar(x=x2, y=rsf_paper['perf%'],
                     name='Perf%=((Pr.At+DY)/(Pr.Ct+RF)-1)',
                     marker_color='rgb(70, 130, 180)')
    trace22 = go.Bar(x=x2, y=rsf_paper['overprice%'],
                     name='OverPrice%=((Pr.At)/(Pr.Ct)-1)',
                     marker_color='rgb(176, 196, 222)')
    trace24 = go.Bar(x=x2, y=rsf_paper['dy12macum%'],
                     name='DY% Ano', marker_color='rgb(135, 206, 250)')
    trace26 = go.Bar(x=x2, y=rsf_paper['p/vpaN'],
                     name='P/VPA', marker_color='rgb(150, 150, 150)')
    trace28 = go.Bar(x=x2, y=rsf_paper['varpatr%'],
                     name='%Var. Patr. Acum',
                     marker_color='rgb(211, 211, 211)')
    data2 = [trace20, trace21, trace22, trace24, trace26, trace28]
    fig2 = go.Figure(data2)
    fig2.update_layout(title='ANÁLISE FIIs PAPEL | Pr.Custo=90%Pr.Atual,\
 DYAno>=3%, Patr.>500M, Neg/dia>1000, P/VPA =< 1.25')
    py.plot(fig2)
    fig2.show()  # to use with Jupyter Notebook
    print(f'N DYs = {rsf_time} meses')
    print(dt_loc)
    sleep(8)
    # rename and move tmp files files generated by plotly
    os.rename('temp-plot.html',
              'C:/Users/maber/OneDrive/Documentos/3_Financas/FIIs/rsf_paper'
              + str(rsf_time) + 'm_' + dt_loc_file+'.html')
#

N DYs = 12 meses
19/11/2020 16:53
